In [2]:
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, Normalizer
from sklearn.svm import SVC
from facenet_pytorch import MTCNN
import torch

In [6]:
 
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = pyplot.imread(filename)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    print(results)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

In [7]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat
 


In [26]:
# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
 
# define filenames
filenames = ['testing_data/ahmed_new.jpg', 'testing_data/ahmed1.1.jpeg', 'testing_data/fouz.jpeg', 'testing_data/fouzan.jpeg','testing_data/ahmed1.2.jpeg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
print(embeddings[0].shape)
print(embeddings[1].shape)
# # define sharon stone
# sharon_id = embeddings[0]
# # verify known photos of sharon
print('Positive Tests')
is_match(embeddings[0], embeddings[4])
# is_match(embeddings[0], embeddings[2])em
# # verify known photos of other people
print('Negative Tests')
is_match(embeddings[0], embeddings[2])

[{'box': [587, 280, 221, 286], 'confidence': 1.0, 'keypoints': {'left_eye': (666, 384), 'right_eye': (767, 387), 'nose': (726, 441), 'mouth_left': (674, 497), 'mouth_right': (760, 498)}}]
[{'box': [504, 697, 169, 298], 'confidence': 0.9475782513618469, 'keypoints': {'left_eye': (621, 822), 'right_eye': (671, 817), 'nose': (670, 888), 'mouth_left': (609, 943), 'mouth_right': (645, 940)}}]
[{'box': [47, 32, 43, 48], 'confidence': 0.9989558458328247, 'keypoints': {'left_eye': (61, 53), 'right_eye': (81, 52), 'nose': (72, 63), 'mouth_left': (63, 71), 'mouth_right': (81, 71)}}]
[{'box': [357, 71, 112, 137], 'confidence': 1.0, 'keypoints': {'left_eye': (389, 123), 'right_eye': (443, 123), 'nose': (415, 147), 'mouth_left': (390, 173), 'mouth_right': (438, 174)}}]
[{'box': [31, 67, 36, 39], 'confidence': 0.9999871253967285, 'keypoints': {'left_eye': (43, 82), 'right_eye': (60, 79), 'nose': (54, 91), 'mouth_left': (46, 98), 'mouth_right': (62, 96)}}]
(2048,)
(2048,)
Positive Tests
>face is NOT 

In [30]:
trainy = [0,0,1,1,0]
model = SVC(kernel='linear', probability=True)
model.fit(embeddings, trainy)
# predict
yhat_train = model.predict(embeddings)
yhat_test = model.predict(embeddings)
# score
score_train = accuracy_score(trainy, yhat_train)
score_test = accuracy_score(trainy, yhat_test)

# summarize
print(f"Accuracy: train={score_train*100}, test={score_test*100}")

Accuracy: train=100.0, test=100.0


In [32]:
yhat_test

array([0, 0, 1, 1, 0])